In [83]:
import numpy as np
import matplotlib.pylab as plt
from scipy import stats
from scipy.spatial.distance import pdist,squareform
import pandas as pd
import os
import itertools
import glob
import math
import time

def return_msa_parsed(align_file):
    uid = []
    sequence = []
    spec = []
    lines = open(align_file, "r")
    for line in lines:
        line = line.rstrip()
        if line[0] == ">":
            uid.append(line[1:])
            spec.append(line[1:].split('/')[0].split('_')[1])
            sequence.append([])
        else:
            sequence[-1].append(line)
    lines.close()
    sequence = [''.join(seq) for seq in sequence]
    print('parsing finished for {} sequences'.format(len(uid)))
    msa_df = pd.DataFrame()
    msa_df['species']=pd.Series(spec,dtype='category')
    msa_df['seqs']=np.arange(len(uid),dtype=np.int)
    print('pd initialized')
    return msa_df,sequence,uid

def write_concat_a2m(msa1_df,sequence_1,uid1,msa2_df,sequence_2,uid2,filename):
    #species_list = np.unique(np.concatenate((msa2_df['species'].values,msa1_df['species'].values)))
#     plt.histogram(np.concatenate((msa2_df['species'].values,msa1_df['species'].values)))
#     plt.savefig(os.path.basename(filename)+'/species_hist.png')         
    species_list = np.intersect1d(msa2_df['species'].unique(),msa1_df['species'].unique())
#     plt.histogram(np.concatenate((msa2_df['species'].values,msa1_df['species'].values)))
#     plt.savefig(os.path.basename(filename)+'/species_hist.png')   
    print('total number of species:{}'.format(len(species_list)))
    start = time.time()
    with open(filename,'w+') as f:
        for spec in species_list:
            #print(spec)
            #print('monomer 1:{} \t monomer 2:{} \t total alignments per specie: {}'.format(len(msa1_df[msa1_df['species']==spec]['seqs'].values),
            #                                                                               len(msa2_df[msa2_df['species']==spec]['seqs'].values),
            #      (len(msa2_df[msa2_df['species']==spec]['seqs'].values)*len(msa1_df[msa1_df['species']==spec]['seqs'].values)))
            #     )
            for prod in (itertools.product(msa1_df[msa1_df['species']==spec]['seqs'].values,msa2_df[msa2_df['species']==spec]['seqs'].values)):
                f.write(uid1[prod[0]]+uid2[prod[1]]+'\n')
                f.write(sequence_1[prod[0]]+sequence_2[prod[1]]+'\n')
    print('time elapsed: {} seconds'.format(time.time()-start))

In [84]:
hpc_df = pd.read_csv('high_precision_complexes.csv',header=None)
for index,row in hpc_df.iterrows():
    aligns = glob.glob('/home/as974/marks/users/kbrock/ecoli_complex/calibration/output/'+row.values[1]+'/align*')
    align_1 = aligns[0]+'/'+row.values[1]+'.a2m'
    align_2 = aligns[1]+'/'+row.values[1]+'.a2m'
    file_dir = 'benchmark/'+row.values[1]
    try: os.mkdir(file_dir)
    except: print(row.values[1] + ' already exists')
    msa1_df,sequence_1,uid1 = return_msa_parsed(align_1)
    msa2_df,sequence_2,uid2 = return_msa_parsed(align_2)
    print('finished parsing individual alignments')
    filename = file_dir+'/concatenation_fixed.a2m'
    write_concat_a2m(msa1_df,sequence_1,uid1,msa2_df,sequence_2,uid2,filename)
    print('finished concatenating alignments')
    del msa1_df
    del msa2_df


allpdb0042 already exists
parsing finished for 152963 sequences
pd initialized
parsing finished for 7656 sequences
pd initialized
finished parsing individual alignments
total number of species:1222
time elapsed: 19.50460696220398 seconds
finished concatenating alignments
